In [2]:
import pandas as pd
import numpy as np
from datasets import Dataset

DATA_PATH = 'data/'

## PURE Dataset

In [3]:
# Load PURE dataset
pure = pd.read_csv(DATA_PATH + 'PURE_train.csv')
tmp = pd.read_csv(DATA_PATH + 'PURE_test.csv')
pure = pd.concat([pure, tmp], axis=0)
tmp = pd.read_csv(DATA_PATH + 'PURE_valid.csv')
pure = pd.concat([pure, tmp], axis=0)

pure['Req/Not Req'] = pure['Req/Not Req'].apply(lambda x: 1 if x == 'Req' else 0)

pure['text'] = pure['Requirement']
pure['label'] = pure['Req/Not Req']
pure = pure.drop(['Unnamed: 0', 'Name of Doc', 'Requirement', 'Req/Not Req'], axis=1)

pure = pure.sample(frac=1).reset_index(drop=True)

print(pure['label'].value_counts())
pure.head(10)

label
1    4145
0    3600
Name: count, dtype: int64


,text,label
0,NPAC SMS shall set a Subscription Version stat...,1
1,To provide a Microsoft Windows application tha...,0
2,The Subscriptions file given in the example wo...,0
3,NPAC SMS shall allow the NPAC SMS Administrato...,1
4,Parameters such as deadline and cost are cruci...,1
5,"As such, it aims for a high level of readabili...",0
6,Contaminated Number - An unavailable number (e...,0
7,The Run/Parse Target menu item shall provide a...,1
8,The Center shall support the following informa...,1
9,"After a few moves, the combination of tiles in...",0


## Dronology Dataset

In [4]:
# Load dronology dataset
dronology = pd.read_csv(DATA_PATH + 'dronology_five_folds/fold_1/train_fold_1.csv')
tmp = pd.read_csv(DATA_PATH + 'dronology_five_folds/fold_1/test_fold_1.csv')
dronology = pd.concat([dronology, tmp], axis=0)

for i in range(2, 6):
    tmp = pd.read_csv(DATA_PATH + 'dronology_five_folds/fold_{}/train_fold_{}.csv'.format(i, i))
    dronology = pd.concat([dronology, tmp], axis=0)
    tmp = pd.read_csv(DATA_PATH + 'dronology_five_folds/fold_{}/test_fold_{}.csv'.format(i, i))
    dronology = pd.concat([dronology, tmp], axis=0)

dronology['text'] = dronology['STR.REQ']
dronology['label'] = dronology['class']
dronology = dronology.drop(['issueid', 'STR.REQ', 'class'], axis=1)

dronology = dronology.drop_duplicates(subset=["text"], keep="first")

print(dronology['label'].value_counts())
dronology.head(10)

label
0    280
1     99
Name: count, dtype: int64


,text,label
0,The MapComponent shall support different types...,1
1,The MissionPlanner shall execute flight plans ...,1
2,The GCS shall transmit the UAV s properties to...,1
3,The GCS shall transmit the UAV s current locat...,1
4,The GCS shall report newly connected UAVs to t...,1
5,When the GCS receives a UAV command from the G...,1
6,When the connection to the GCS from the GCSMid...,1
7,The GCSMiddleware shall forward commands sent ...,1
8,The GCSMiddleware shall handle state messages ...,1
9,The GCSMiddleware shall follow a predefined se...,1


In [5]:
# Merge two datasets
data = pd.concat([pure, dronology], axis=0)
print(len(data))
data['label'].value_counts()

8124


label
1    4244
0    3880
Name: count, dtype: int64

In [6]:
data['text_length'] = data['text'].apply(lambda x: len(x))
np.mean(data['text_length'])

133.74766125061547

In [7]:
data = Dataset.from_pandas(data)
data = data.train_test_split(test_size=0.3)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'text_length', '__index_level_0__'],
        num_rows: 5686
    })
    test: Dataset({
        features: ['text', 'label', 'text_length', '__index_level_0__'],
        num_rows: 2438
    })
})

In [9]:
data['train'][190]

{'text': 'NPAC SMS shall provide post‑collection audit analysis tools that can produce detailed reports on data items relating to system intrusions.',
 'label': 1,
 'text_length': 138,
 '__index_level_0__': 6850}